In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

https://docs.llamaindex.ai/en/stable/examples/agent/multi_document_agents/

# Multi-Document Agents

In this guide, you learn towards setting up an agent that can effectively answer different types of questions over a larger set of documetns. 

These questions include the following

* QA over a specific doc
* QA comparing different docs
* Summaries over a specific doc
* Comparing summaries between different docs

We do this with the following architecture:

* setup a "document agent" over each Document: each doc agent can do QA/summarization within its doc
* setup a top-level agent over this set of document agents. Do tool retrieval and then do CoT over the set of tools to answer a question.


## Setup and download data

In this section, we'll define imports and then download Wikipedia articles about different cities. Each article is stored separately.

We load in 18 ciites - this is not quite at the level of "hundreds" of documents but its still large enough to warrant some top-level document retrieval!



```sh
pip install llama-index-agent-openai
pip install llama-index-embeddings-openai
pip install llama-index-llms-openai
```

In [2]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleKeywordTableIndex,
    SimpleDirectoryReader,
)
from llama_index.core import SummaryIndex
from llama_index.core.schema import IndexNode
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.openai import OpenAI
from llama_index.core.callbacks import CallbackManager

In [3]:
wiki_titles = [
    "Toronto",
    "Seattle",
    "Chicago",
    "Boston",
    "Houston",
    "Tokyo",
    "Berlin",
    "Lisbon",
    "Paris",
    "London",
    "Atlanta",
    "Munich",
    "Shanghai",
    "Beijing",
    "Copenhagen",
    "Moscow",
    "Cairo",
    "Karachi",
    "Prague",
    "Seoul",
    "Albuquerque",
    "Montreal",
    "Oslo"
]

In [4]:
from pathlib import Path

import requests

for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            # 'exintro': True,
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    data_path = Path("data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w") as fp:
        fp.write(wiki_text)

In [5]:
# Load all wiki documents
city_docs = {}
for wiki_title in wiki_titles:
    city_docs[wiki_title] = SimpleDirectoryReader(
        input_files=[f"data/{wiki_title}.txt"]
    ).load_data()

## Define glocal LLM and  Embeddings

In [6]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

#Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
#Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

Settings.llm = OpenAI(temperature=0, model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")


In [8]:
#help(OpenAIEmbedding)

# Building Multi-Document Agents

In this section we show you how to construct the multi-document agent. We first build a document ech for each document, and then define the top-level parent agent with an object index.

## Build Document Agent for each Document

In this sectoin we define "document agents" for each document.

We define both a vector index (for semantic search) and summary index (for summarization) for each document. The two query engines are then converted into tools that are passed to an OpenAI functoin calling agent.

This document agent can dynamically choose to perform semantic search or summarization wtihin a given document.

We create a seprate document agent for each city.

In [27]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.core import load_index_from_storage, StorageContext
from llama_index.core.node_parser import SentenceSplitter
import os

node_parser = SentenceSplitter()

# Build agents dictionary
agents = {}
query_engines = {}

# this is for the baseline
all_nodes = []

for idx, wiki_title in enumerate(wiki_titles):
    nodes = node_parser.get_nodes_from_documents(city_docs[wiki_title])
    all_nodes.extend(nodes)

    if not os.path.exists(f"./data/{wiki_title}"):
        # build vector index
        vector_index = VectorStoreIndex(nodes)
        vector_index.storage_context.persist(
            persist_dir=f"./data/{wiki_title}"
        )
    else:
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=f"./data/{wiki_title}"),
        )

    # build summary index
    summary_index = SummaryIndex(nodes)
    # define query engines
    vector_query_engine = vector_index.as_query_engine(llm=Settings.llm)
    summary_query_engine = summary_index.as_query_engine(llm=Settings.llm)

    # define tools
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name="vector_tool",
                description=(
                    "Useful for questions related to specific aspects of"
                    f" {wiki_title} (e.g. the history, arts and culture,"
                    " sports, demographics, or more)."
                ),
            ),
        ),
        QueryEngineTool(
            query_engine=summary_query_engine,
            metadata=ToolMetadata(
                name="summary_tool",
                description=(
                    "Useful for any requests that require a holistic summary"
                    f" of EVERYTHING about {wiki_title}. For questions about"
                    " more specific sections, please use the vector_tool."
                ),
            ),
        ),
    ]

    # build agent
    function_llm = OpenAI(model="gpt-4")
    agent = OpenAIAgent.from_tools(
        query_engine_tools,
        llm=function_llm,
        verbose=True,
        system_prompt=f"""\
You are a specialized agent designed to answer queries about {wiki_title}.
You must ALWAYS use at least one of the tools provided when answering a question; do NOT rely on prior knowledge.\
""",
    )

    agents[wiki_title] = agent
    query_engines[wiki_title] = vector_index.as_query_engine(
        similarity_top_k=20
    )

# Build Retriever-Enabled OpenAI Agent

We build a top-level agent that can orchestrate across the different document agents to answer any user query. 

This agent takes in all document agents as tools. This specific agent `RetrieverOpenAIAgent` performs tool retrieval before tool use (unlike a default agent that tries to pus all tools in the prompt)

Here we use a top-k retriever, but we encourage you to customizet he tool retriever method!

In [28]:
# define tool for each document agent
all_tools = []
for wiki_title in wiki_titles:
    wiki_summary = (
        f"This content contains Wikipedia articles about {wiki_title}. Use"
        f" this tool if you want to answer any questions about {wiki_title}.\n"
    )
    doc_tool = QueryEngineTool(
        query_engine=agents[wiki_title],
        metadata=ToolMetadata(
            name=f"tool_{wiki_title}",
            description=wiki_summary,
        ),
    )
    all_tools.append(doc_tool)

In [29]:
# define an "object" index and retriever over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [30]:
from llama_index.agent.openai import OpenAIAgent

top_agent = OpenAIAgent.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=5),
    system_prompt=""" \
You are an agent designed to answer queries about a set of given cities.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    verbose=True,
)

# Define Baseline Vector STore Index

As a point of comparison, we define a "naive" RAG pipeline which dumps all docs into a single vector index collection.

We set the top_k = 4.

In [32]:
base_index = VectorStoreIndex(all_nodes)
base_query_engine = base_index.as_query_engine(similarity_top_k=20)

# Running Example Queries

Let's run some example queries, ranging from QA / summaries over a single document to QA / Summarization over multiple documents

In [33]:
# should use Boston agent -> vector tool
response = top_agent.query("Tell me about the arts and culture in Boston.")

Added user message to memory: Tell me about the arts and culture in Boston.
=== Calling Function ===
Calling function: tool_Boston with args: {"input":"arts and culture"}
Added user message to memory: arts and culture
=== Calling Function ===
Calling function: vector_tool with args: {
  "input": "arts and culture"
}
Got output: Boston boasts a rich arts and culture scene, featuring numerous art museums and galleries such as the Museum of Fine Arts and the Isabella Stewart Gardner Museum. The Institute of Contemporary Art, located in a modern building in the Seaport District, adds to the city's contemporary art offerings. The South End Art and Design District (SoWa) and Newbury Street are popular destinations for art galleries.

The city has a strong literary heritage, often referred to as the "Athens of America" due to its historical significance in American literature, with notable writers like Ralph Waldo Emerson and Henry Wadsworth Longfellow having roots in Boston. The Boston Publi

In [34]:
print(response)

Boston is renowned for its vibrant arts and culture scene. The city boasts numerous art museums and galleries, including the Museum of Fine Arts and the Isabella Stewart Gardner Museum. The Institute of Contemporary Art, located in the Seaport District, is a significant contributor to contemporary art offerings. Art enthusiasts also frequent the South End Art and Design District (SoWa) and Newbury Street for their array of galleries.

The city's literary heritage is noteworthy, often referred to as the "Athens of America" due to its historical significance in American literature. Notable writers like Ralph Waldo Emerson and Henry Wadsworth Longfellow have roots in Boston. The Boston Public Library, the first free library in the United States, continues to support the city's literary culture.

In music, the Boston Symphony Orchestra is one of the top orchestras in the country, and Symphony Hall, known for its excellent acoustics, hosts various performances, including those by the Boston

In [35]:
# baseline
response = base_query_engine.query(
    "Tell me about the arts and culture in Boston"
)
print(str(response))

Boston boasts a rich arts and culture scene, deeply rooted in its historical significance and diverse community. The city is known for its vibrant literary culture, often referred to as the "Athens of America," with a legacy that includes renowned writers such as Ralph Waldo Emerson and Henry Wadsworth Longfellow. The Old Corner Bookstore is celebrated as a historic literary meeting place.

In the realm of music, Boston is home to prestigious institutions like the Boston Symphony Orchestra, recognized as one of the world's best orchestras, and Symphony Hall, a premier venue for classical music. The city also supports a variety of performing arts organizations, including the Boston Ballet and the Boston Lyric Opera Company. The Theater District features numerous theaters, showcasing a range of performances from Broadway shows to local productions.

Boston hosts several major annual cultural events, such as the Boston Book Festival and First Night on New Year's Eve, which celebrate the c

In [36]:
# should use Houston agent -> vector tool
response = top_agent.query(
    "Give me a summary of all the positive aspects of Seoul"
)

Added user message to memory: Give me a summary of all the positive aspects of Seoul
=== Calling Function ===
Calling function: tool_Seoul with args: {"input":"positive aspects of Seoul"}
Added user message to memory: positive aspects of Seoul
=== Calling Function ===
Calling function: vector_tool with args: {
  "input": "positive aspects of Seoul"
}
Got output: Seoul boasts several positive aspects, including:

1. **Economic Hub**: It serves as the business and financial center of South Korea, contributing significantly to the national GDP and hosting numerous Fortune Global 500 companies.

2. **High Quality of Life**: The city has been recognized for its livability and quality of life, ranking highly in global assessments.

3. **Cultural Richness**: Seoul is home to five UNESCO World Heritage Sites and has a vibrant cultural scene, including traditional and modern art markets.

4. **Modern Infrastructure**: The city features impressive modern architecture and infrastructure, includin

In [37]:
# should use Houston agent -> vector tool
response = base_query_engine.query(
    "Give me a summary of all the positive aspects of Seoul"
)

In [38]:
print(str(response))

Seoul is recognized for its high quality of life, having been rated as Asia's most livable city and the second-highest globally in 2015. The city boasts a technologically advanced infrastructure, being the world's "most wired city" with exceptional internet connectivity and high-speed broadband access. It is home to major global companies, including 15 Fortune Global 500 firms, and has significant economic influence, contributing a substantial portion of South Korea's GDP.

Culturally, Seoul is rich in history and heritage, featuring five UNESCO World Heritage Sites and numerous museums, including the National Museum of Korea. The city is a hub for the arts, hosting various festivals and events that celebrate music, dance, and traditional performances. Its vibrant neighborhoods offer a mix of modern and traditional experiences, with bustling markets and shopping districts.

Seoul's transportation system is well-developed, with an extensive subway network and bus services that facilitat

In [ ]:
# should use Houston agent -> vector tool
response = top_agent.query(
    "Give me a detailed summary of Seoul"
)

Added user message to memory: Give me a detailed summary of Seoul
=== Calling Function ===
Calling function: tool_Seoul with args: {"input":"detailed summary"}
Added user message to memory: detailed summary
=== Calling Function ===
Calling function: summary_tool with args: {
  "input": "Seoul"
}
Got output: Seoul, officially known as Seoul Special Metropolitan City, is the capital and largest city of South Korea. It is part of the broader Seoul Capital Area, which includes Gyeonggi province and Incheon Metropolitan City, making it one of the world's largest metropolitan economies. The city has a rich history dating back to 18 BC when it was founded by the Baekje kingdom. Over the centuries, it has served as the capital during various dynasties, including the Joseon dynasty, and has undergone significant transformations, especially during the Japanese occupation and the Korean War.

As of 2020, Seoul's population was approximately 9.97 million, having peaked at over 10 million in 2014. 